<a href="https://colab.research.google.com/github/krugis/alitolgaeskici/blob/main/crawl_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
## reference https://nariyoo.com/python-how-to-run-selenium-in-google-colab/
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,860 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,519 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu jammy-

In [3]:
!pip install chromedriver-autoinstaller


In [ ]:
!pip install prettytable  # Install prettytable for table formatting

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import chromedriver_autoinstaller

import sys

sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# Add arguments to disable images and unnecessary resources
chrome_options.add_argument('--disable-images')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.page_load_strategy = 'eager'  # Use 'eager' page load strategy

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)


# Function to get tweets from a specific user
def get_tweets(username, num_tweets=5):
    url = f"https://twitter.com/{username}"
    driver.get(url)
    time.sleep(2)  # Reduced initial wait time

    all_tweets = []  # Store all extracted tweets
    seen_tweets = set()  # Store tweet texts to avoid duplicates
    scroll_attempts = 0  # Track scroll attempts
    max_scroll_attempts = 10  # Maximum number of scroll attempts

    while len(all_tweets) < num_tweets and scroll_attempts < max_scroll_attempts:
        # Scroll down to load more tweets (before parsing)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)  # Wait for new tweets to load
        scroll_attempts += 1  # Increment scroll attempts

        # Parse the page source once outside the loop
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tweets = soup.find_all('article', {'data-testid': 'tweet'})

        # Process tweets using list comprehension and JavaScript execution once for all tweets
        new_tweets = [
            (
                tweet.find('div', {'data-testid': 'tweetText'}).text if tweet.find('div', {'data-testid': 'tweetText'}) else None,
                driver.execute_script(
                    """
                    const tweetHTML = arguments[0];
                    const tweet = document.createElement('div');
                    tweet.innerHTML = tweetHTML;

                    return {
                        comment_count: tweet.querySelector('[data-testid="reply"]')?.getAttribute('aria-label')?.split(' ')[0] || '0',
                        retweet_count: tweet.querySelector('[data-testid="retweet"]')?.getAttribute('aria-label')?.split(' ')[0] || '0',
                        like_count: tweet.querySelector('[data-testid="like"]')?.getAttribute('aria-label')?.split(' ')[0] || '0',
                    };
                    """,
                    str(tweet),  # Pass the tweet's outer HTML as a string
                )
            )
            for tweet in tweets
            if tweet.find('div', {'data-testid': 'tweetText'}) is not None and tweet.find('div', {'data-testid': 'tweetText'}).text not in seen_tweets
        ]

        # Extend the list of tweets and update the seen tweets set
        all_tweets.extend([(tweet_text, tweet_data['comment_count'], tweet_data['retweet_count'], tweet_data['like_count'])
                            for tweet_text, tweet_data in new_tweets])
        seen_tweets.update([tweet_text for tweet_text, _ in new_tweets])

    # Print or return the extracted tweets
    for tweet in all_tweets:
        print("Tweet:", tweet[0])
        print("Comments:", tweet[1])
        print("Retweets:", tweet[2])
        print("Likes:", tweet[3])
        print("-----")


# Example usage
get_tweets('GoatIndexAI', num_tweets=10)

# Close the WebDriver
driver.quit()

Tweet: Introducing Solana AI Ecosystem Airdrop by @GoatIndexAI

A movement to rally the Solana AI Ecosystem to grow 100x from here.

Launching on @MeteoraAG on Jan 16th.
Comments: 387
Retweets: 505
Likes: 1529
-----
Tweet:  BREAKING: GoatIndex Agent can now detect ANY new CAs posted by Trump family/cabal!

If you had bought $TRUMP the second its CA was posted on his X account, you could have made 30x your money.

You really think other Trump family or cabal members won’t drop their own CAs? 
Comments: 28
Retweets: 33
Likes: 182
-----
Tweet: After tonight, 50M new users are onboarding to @solana and @MeteoraAG.

@solana to $1T
@MeteoraAG to $100B
Comments: 5
Retweets: 7
Likes: 54
-----
Tweet: Good News: Our Twitter CA Monitoring tool (with image recognition capability) works for the Trump Tweet

Bad News: $TRUMP is not a Solana AI Project so we did not monitor his Twitter Profile
Comments: 5
Retweets: 6
Likes: 61
-----


In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import chromedriver_autoinstaller
import sys


# Set up for running selenium in Google Colab (only needed for Colab)
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# Add arguments to disable images and unnecessary resources
chrome_options.add_argument('--disable-images')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.page_load_strategy = 'eager'  # Use 'eager' page load strategy

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)


# Function to get tweets from a specific user
def get_tweets(username, num_tweets):
    url = f"https://twitter.com/{username}"
    driver.get(url)

    all_tweets = []
    seen_tweets = set()

    # Wait for the initial tweets to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'article[data-testid="tweet"]'))
    )

    # Initial scroll to load more tweets
    driver.execute_script("window.scrollTo(0, 500);")
    time.sleep(1)

    while len(all_tweets) < num_tweets:
        # Get all tweets currently loaded on the page
        tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')

        # Process tweets and extract details
        for tweet in tweets:
            try:
                tweet_text_element = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')
                tweet_text = tweet_text_element.text
            except:
                continue

            if tweet_text not in seen_tweets:
                seen_tweets.add(tweet_text)

                tweet_details = {
                    "comment_count": tweet.find_element(By.CSS_SELECTOR, '[data-testid="reply"]').get_attribute('aria-label').split(' ')[0] if tweet.find_elements(By.CSS_SELECTOR, '[data-testid="reply"]') else '0',
                    "retweet_count": tweet.find_element(By.CSS_SELECTOR, '[data-testid="retweet"]').get_attribute('aria-label').split(' ')[0] if tweet.find_elements(By.CSS_SELECTOR, '[data-testid="retweet"]') else '0',
                    "like_count": tweet.find_element(By.CSS_SELECTOR, '[data-testid="like"]').get_attribute('aria-label').split(' ')[0] if tweet.find_elements(By.CSS_SELECTOR, '[data-testid="like"]') else '0'
                }

                all_tweets.append((tweet_text, tweet_details["comment_count"], tweet_details["retweet_count"], tweet_details["like_count"]))

                if len(all_tweets) >= num_tweets:
                    break

        if len(all_tweets) >= num_tweets:
            break

        # Scroll down incrementally
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(1)

    # Print extracted tweets
    for tweet in all_tweets[:num_tweets]:
        print("Tweet:", tweet[0])
        print("Comments:", tweet[1])
        print("Retweets:", tweet[2])
        print("Likes:", tweet[3])
        print("-----")

    print(f"Total number of tweets extracted: {len(all_tweets)}")


# Example usage
get_tweets('GoatIndexAI', num_tweets=10)

# Close the WebDriver
driver.quit()

Tweet: Introducing Solana AI Ecosystem Airdrop by 
@GoatIndexAI
A movement to rally the Solana AI Ecosystem to grow 100x from here.

Launching on 
@MeteoraAG
 on Jan 16th.
Comments: 390
Retweets: 504
Likes: 1526
-----
Tweet: Solana AI Index Recap (Jan 20)

Total Projects: 529
Projects with tokens: 301
Total Market Cap: $6B (-0%)
Trading Volume (24H): $516M (-34%)

 Top Projects (6H)
- 
@PunkyAi
 
- 
@aixbt_agent
 ($640M MC, +17.8%)
- 
Comments: 5
Retweets: 5
Likes: 34
-----
Tweet: A great testament to our Twitter CA Tracking feature!

Link in bio

$AIAI
Comments: 7
Retweets: 9
Likes: 71
-----
Tweet: BREAKING: The GoatIndex Agent called this the same minute 
@MELANIATRUMP
 posted the CA

Join our TG chat to get realtime alert of any CA posted by the Trump family/cabal.

Link in bio
Comments: 58
Retweets: 67
Likes: 171
-----
Tweet: gud tech: 
Comments: 1
Retweets: 3
Likes: 14
-----
Tweet: TG Link
Comments: 0
Retweets: 2
Likes: 11
-----
Tweet: This is a playbook of how I prefer to play re

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import chromedriver_autoinstaller
import sys


# Set up for running selenium in Google Colab (only needed for Colab)
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# Add arguments to disable images and unnecessary resources
chrome_options.add_argument('--disable-images')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.page_load_strategy = 'eager'  # Use 'eager' page load strategy

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)


# Function to get tweets from a specific user
def get_tweets(username, num_tweets):
    url = f"https://twitter.com/{username}"
    driver.get(url)

    all_tweets = []
    seen_tweets = set()

    # Wait for the initial tweets to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'article[data-testid="tweet"]'))
    )

    # Initial scroll to load more tweets
    driver.execute_script("window.scrollTo(0, 500);")
    time.sleep(1)

    while len(all_tweets) < num_tweets:
        # Get all tweets currently loaded on the page
        tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')

        # Process tweets and extract details
        for tweet in tweets:
            try:
                tweet_text_element = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')
                tweet_text = tweet_text_element.text
            except:
                continue

            if tweet_text not in seen_tweets:
                seen_tweets.add(tweet_text)

                # Extract tweet details, including views
                tweet_details = {
                    "comment_count": tweet.find_element(By.CSS_SELECTOR, '[data-testid="reply"]').get_attribute('aria-label').split(' ')[0] if tweet.find_elements(By.CSS_SELECTOR, '[data-testid="reply"]') else '0',
                    "retweet_count": tweet.find_element(By.CSS_SELECTOR, '[data-testid="retweet"]').get_attribute('aria-label').split(' ')[0] if tweet.find_elements(By.CSS_SELECTOR, '[data-testid="retweet"]') else '0',
                    "like_count": tweet.find_element(By.CSS_SELECTOR, '[data-testid="like"]').get_attribute('aria-label').split(' ')[0] if tweet.find_elements(By.CSS_SELECTOR, '[data-testid="like"]') else '0',
                    # Extract views from aria-label
                    "view_count": tweet.find_element(By.CSS_SELECTOR, 'a[aria-label*="views"]').get_attribute('aria-label').split(' ')[0] if tweet.find_elements(By.CSS_SELECTOR, 'a[aria-label*="views"]') else '0'
                }

                all_tweets.append((tweet_text, tweet_details["comment_count"], tweet_details["retweet_count"], tweet_details["like_count"], tweet_details["view_count"]))

                if len(all_tweets) >= num_tweets:
                    break

        if len(all_tweets) >= num_tweets:
            break

        # Scroll down incrementally
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(1)

    # Print extracted tweets with views
    for tweet in all_tweets[:num_tweets]:
        print("Tweet:", tweet[0])
        print("Comments:", tweet[1])
        print("Retweets:", tweet[2])
        print("Likes:", tweet[3])
        print("Views:", tweet[4])
        print("-----")

    print(f"Total number of tweets extracted: {len(all_tweets)}")


# Example usage
get_tweets('GoatIndexAI', num_tweets=10)

# Close the WebDriver
driver.quit()

Tweet: Introducing Solana AI Ecosystem Airdrop by 
@GoatIndexAI
A movement to rally the Solana AI Ecosystem to grow 100x from here.

Launching on 
@MeteoraAG
 on Jan 16th.
Comments: 390
Retweets: 503
Likes: 1524
Views: 538904
-----
Tweet: Solana AI Index Recap (Jan 20)

Total Projects: 529
Projects with tokens: 301
Total Market Cap: $6B (-0%)
Trading Volume (24H): $516M (-34%)

 Top Projects (6H)
- 
@PunkyAi
 
- 
@aixbt_agent
 ($640M MC, +17.8%)
- 
Comments: 6
Retweets: 5
Likes: 41
Views: 7209
-----
Tweet: A great testament to our Twitter CA Tracking feature!

Link in bio

$AIAI
Comments: 7
Retweets: 9
Likes: 71
Views: 13368
-----
Tweet: BREAKING: The GoatIndex Agent called this the same minute 
@MELANIATRUMP
 posted the CA

Join our TG chat to get realtime alert of any CA posted by the Trump family/cabal.

Link in bio
Comments: 58
Retweets: 67
Likes: 171
Views: 32382
-----
Tweet: gud tech: 
Comments: 1
Retweets: 3
Likes: 14
Views: 8834
-----
Tweet: TG Link
Comments: 0
Retweets: 2
Like

In [25]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import chromedriver_autoinstaller
import sys
from prettytable import PrettyTable  # Import PrettyTable for table formatting
import re # Import re for regular expression matching
from datetime import datetime, timezone  # Import datetime and timezone for timestamp conversion


# Set up for running selenium in Google Colab (only needed for Colab)
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# Add arguments to disable images and unnecessary resources
chrome_options.add_argument('--disable-images')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.page_load_strategy = 'eager'  # Use 'eager' page load strategy

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)


# Function to get tweets from a specific user
def get_tweets(username, num_tweets):
    url = f"https://twitter.com/{username}"
    driver.get(url)

    all_tweets = []
    seen_tweets = set()

    # Wait for the initial tweets to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'article[data-testid="tweet"]'))
    )

    # Initial scroll to load more tweets
    driver.execute_script("window.scrollTo(0, 500);")
    time.sleep(1)

    while len(all_tweets) < num_tweets:
        # Get all tweets currently loaded on the page
        tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')

        # Process tweets and extract details
        for tweet in tweets:
            try:
                tweet_text_element = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')
                tweet_text = tweet_text_element.text
            except:
                continue

            if tweet_text not in seen_tweets:
                seen_tweets.add(tweet_text)

                # Extract tweet details, including views
                tweet_details = {
                    "comment_count": tweet.find_element(By.CSS_SELECTOR, '[data-testid="reply"]').get_attribute('aria-label').split(' ')[0] if tweet.find_elements(By.CSS_SELECTOR, '[data-testid="reply"]') else '0',
                    "retweet_count": tweet.find_element(By.CSS_SELECTOR, '[data-testid="retweet"]').get_attribute('aria-label').split(' ')[0] if tweet.find_elements(By.CSS_SELECTOR, '[data-testid="retweet"]') else '0',
                    "like_count": tweet.find_element(By.CSS_SELECTOR, '[data-testid="like"]').get_attribute('aria-label').split(' ')[0] if tweet.find_elements(By.CSS_SELECTOR, '[data-testid="like"]') else '0',
                    "view_count": tweet.find_element(By.CSS_SELECTOR, 'a[aria-label*="views"]').get_attribute('aria-label').split(' ')[0] if tweet.find_elements(By.CSS_SELECTOR, 'a[aria-label*="views"]') else '0'
                }

                # Extract tweet ID, timestamp, and date using regular expressions
                tweet_link_element = tweet.find_element(By.CSS_SELECTOR, 'a[href*="/status/"]')  # Find the element with the tweet link
                tweet_link = tweet_link_element.get_attribute('href')  # Get the tweet link
                tweet_id_match = re.search(r'/status/(\d+)', tweet_link)  # Extract tweet ID using regex
                tweet_id = tweet_id_match.group(1) if tweet_id_match else None  # Get the tweet ID if found

                timestamp_element = tweet.find_element(By.CSS_SELECTOR, 'time')
                timestamp = timestamp_element.get_attribute('datetime')

                # Convert timestamp to Linux format and desired date format
                dt = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S.%fZ')
                dt = dt.replace(tzinfo=timezone.utc).astimezone(timezone.utc) # Set timezone to UTC
                linux_timestamp = int(dt.timestamp())
                date = dt.strftime('%y-%m-%d-%H-%M-%S') # Format date as yy-mm-dd-hh-mm:ss

                # Use username variable as account name
                all_tweets.append((username, tweet_id, linux_timestamp, date, tweet_text[:15] + "..." if len(tweet_text) > 15 else tweet_text, # Shorten to 15 characters
                                   tweet_details["comment_count"], tweet_details["retweet_count"], tweet_details["like_count"], tweet_details["view_count"]))


                if len(all_tweets) >= num_tweets:
                    break

        if len(all_tweets) >= num_tweets:
            break

        # Scroll down incrementally
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(1)

    # Display extracted tweets in a table
    table = PrettyTable()
    table.field_names = ["Account Name", "Twitter ID", "Timestamp", "Date", "Tweet", "Comments", "Retweets", "Likes", "Views"]

    for tweet in all_tweets[:num_tweets]:
        table.add_row(tweet)  # Add each tweet as a row to the table

    print(table)  # Print the formatted table
    print(f"Total number of tweets extracted: {len(all_tweets)}")


# Example usage
get_tweets('GoatIndexAI', num_tweets=5)

# Close the WebDriver
driver.quit()

+--------------+---------------------+------------+-------------------+--------------------+----------+----------+-------+--------+
| Account Name |      Twitter ID     | Timestamp  |        Date       |       Tweet        | Comments | Retweets | Likes | Views  |
+--------------+---------------------+------------+-------------------+--------------------+----------+----------+-------+--------+
| GoatIndexAI  | 1879559052298199390 | 1736956773 | 25-01-15-15-59-33 | Introducing Sol... |   390    |   503    |  1527 | 539079 |
| GoatIndexAI  | 1881498730006020449 | 1737419228 | 25-01-21-00-27-08 | Solana AI Index... |    6     |    5     |   43  |  7850  |
| GoatIndexAI  | 1881226244441915516 | 1737354262 | 25-01-20-06-24-22 | A great testame... |    7     |    9     |   71  | 13416  |
| GoatIndexAI  | 1881090116808314896 | 1737321807 | 25-01-19-21-23-27 | BREAKING: The G... |    58    |    67    |  172  | 32436  |
| GoatIndexAI  | 1881092392507613305 | 1737322350 | 25-01-19-21-32-30 |     